## 発表内容

最近日本語対応したAmazon KendraはAmazon S3やAmazon RDS、Salesforceといったリポジトリのコンテンツを一元管理されたインデックスに簡単に集約できます。これにより企業の様々なデータを素早く検索できます。そして、Amazon Comprehendはドキュメント内のテキストの解析を行うことができます。この2つのサービスを組み合わせた文書検索と検索された文書の解析についてデモを交えてご紹介します。

----

## 概要

- Wikipediaの文章をComprehendで分析させてみる（図②）
- KendraにWikipediaの文章を取り込んで日本語検索してみる（図③、④、⑤）

この二つのサービスを組み合わせることで、Comprehendで文書から抽出したキーワードをKendraの絞り込み検索の時に使うといったような応用も可能です。

----

## アーキテクチャ

![](./architecture_kendra_comprehend.png)

----
## Kendraの利用手順

![](./kendra_flow.png)

----

## デモの流れ

2. Wikipediaから取得したファイルをディレクトリに配置する & S3へのデータ配置（図①、③）
3. Kendraインデックスをプロビジョニングします
4. Comprehendを用いたエンティティ抽出（図②）
6. データソースのsync（図④）
7. 検索！（図⑤）

## 2. Wikipediaから取得したファイルをディレクトリに配置する & S3へのデータ配置（図①、③）

In [ ]:
!mkdir -p Data Meta && mkdir -p Meta/Data && echo 0 > .metalog

!cat aws.txt && echo '----------'
# aws.txtに書かれた情報をWikipediaからcurlしてくる
!sh ./getpages.sh

In [ ]:
# 下準備

# バケット作成
!sh mb.sh

In [ ]:
# 今回のデモではS3をデータソースとしてKendraで検索をしたい。そのためにS3にWikipediaの情報をアップロードする。  

!sh upload.sh > /dev/null

## 3. Kendraインデックスをプロビジョニングします

新しい Kendraインデックス の作成には約15分ほどかかります。（3分クッキング方式でデモ実施します）  
デモでは画面上でインデックスとデータソースを作成する手順をご説明します。  

## 4. Comprehendを用いたエンティティ抽出（図②）

Comprehendのデモ  

[Wikipediaの文章](https://ja.wikipedia.org/wiki/Amazon_Web_Services)を取り込んでみる。 コンソールは[こちら](https://console.aws.amazon.com/comprehend/v2/home?region=us-east-1#home)

----

Wikipediaから取得してきたファイルを対象として、SDKを使ってComprehendでエンティティ抽出を行ってみる。

ファイルを逐次読み込み -> Comprehendのdetect_entitiesメソッドでエンティティを取得 -> 整形してjson化してファイルに格納

```python
# ファイル読み込み
    doc_text = open(fname, 'r', encoding="utf-8").read()
...
# detect_entitiesメソッドでエンティティを取得
    for i in range(0, len(doc_text), compre_text_size):
        try:
            entities = client.detect_entities(Text=doc_text[i:i+compre_text_size], LanguageCode='ja')
...
# json化
    attributes["Attributes"] = metadata
    print(json.dumps(attributes, sort_keys=True, indent=4))
```

In [ ]:
# 1ファイルでエンティティデータ抽出をスクリプトで試してみる
#  元のWikipediaページ
#    https://ja.wikipedia.org/wiki/Amazon_Web_Services

!python3 getpagemetadata.py Data/Amazon_Web_Services.html

## ComprehendとKendraの組み合わせ例

![](./filtersearch.png)

----

## データソースのsyncと検索（図④、⑤）

https://console.aws.amazon.com/kendra/home?region=us-east-1#indexes